# Convolutional Neural Network

#### Note: download tensorflow and pillow first

In [1]:
!pip install tensorflow

### Importing the libraries

In [2]:
!pip install scipy

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
# to avoid overfitting 
train_datagen = ImageDataGenerator(rescale = 1./255,  #feature scaling to all pixels to [0,1]
                                   shear_range = 0.2, #transformations
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set', #path
                                                 target_size = (64, 64), #training size
                                                 batch_size = 32,        #images taking each time
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
#colored image in RGB, it's 64x64 as defined above, so it's 64x64x3
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\src\my\uwf-idc6146-deep-learning\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #pool_size is the dimension of pooling matrix

### Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) # input shape is deleted
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #number of hidden neurons

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #binary output for cats or dogs

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


C:\src\my\uwf-idc6146-deep-learning\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 65ms/step - accuracy: 0.5442 - loss: 0.6967 - val_accuracy: 0.6405 - val_loss: 0.6292
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.6382 - loss: 0.6432 - val_accuracy: 0.6810 - val_loss: 0.6104
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.6786 - loss: 0.5985 - val_accuracy: 0.7095 - val_loss: 0.5849
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 62ms/step - accuracy: 0.7324 - loss: 0.5339 - val_accuracy: 0.7220 - val_loss: 0.5504
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 62ms/step - accuracy: 0.7493 - loss: 0.5125 - val_accuracy: 0.7540 - val_loss: 0.5290
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 65ms/step - accuracy: 0.7596 - loss: 0.4971 - val_accuracy: 0.7760 - val_loss: 0.4921
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 67ms/step - accuracy: 0.7762 - loss: 0.4689 - val_accuracy: 0.7395 - val_loss: 0.5552
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 67ms/step - accuracy: 0.7773 - loss: 0.4619 - val_accurac

## Part 4 - Making a single prediction

In [16]:
def make_prediction(path: str, target_size = (64, 64)) -> str:
    import numpy as np
    from tensorflow.keras.preprocessing import image
    
    test_image = image.load_img(path, target_size=target_size)
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    
    return 'dog' if result[0][0] == 1 else 'cat'

image_path = 'dataset/single_prediction/cat_or_dog_1.jpg'
prediction = make_prediction(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


In [17]:
print(prediction)

dog


## Problem 2b

In [18]:
image_path = 'dataset/single_prediction/cat_or_dog_2.jpg'
prediction = make_prediction(image_path)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
cat


## Problem 2c

In [19]:
!pip install colorama

In [20]:
def get_files_by_extension(folder_path:str, file_extension:str) -> list[str]:
    from pathlib import Path
    path = Path(folder_path)
    return [str(file) for file in path.glob(f'*{file_extension}')]
    
def get_random_file(files:list[str]) -> str:
    import random
    index = random.randint(0, len(files) - 1)
    return files[index]

def try_predict_animal_from_file(file_path: str):
    from colorama import Fore, Style
    
    actual = 'cat' if 'cat' in file_path else 'dog'
    prediction = make_prediction(file_path)
    color = Fore.GREEN if prediction == actual else Fore.RED
    
    print(f"{color}File: {file_path}; Prediction: {prediction}; Actual: {actual}{Style.RESET_ALL}")
    
def try_predict_animals_from_files(files: list[str], iterations=10):
    for iteration in range(iterations):
        random_file = get_random_file(files)
        try_predict_animal_from_file(random_file)


## Try to predict cats

In [21]:
file_extension = '.jpg'
root_path = 'dataset/test_set/cats'
files = get_files_by_extension(root_path, file_extension)

try_predict_animals_from_files(files)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
File: dataset\test_set\cats\cat.4814.jpg; Prediction: cat; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
File: dataset\test_set\cats\cat.4990.jpg; Prediction: dog; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
File: dataset\test_set\cats\cat.4588.jpg; Prediction: cat; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
File: dataset\test_set\cats\cat.4590.jpg; Prediction: dog; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
File: dataset\test_set\cats\cat.4792.jpg; Prediction: cat; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
File: dataset\test_set\cats\cat.4264.jpg; Prediction: dog; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
File: dataset\test_set\cats\cat.4397.jpg; Prediction: dog; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
File: dataset\test_set\cats\cat.4729.jpg; Prediction: cat; Actual: cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
File: dataset\test_set\cats\cat.4589.jpg; Prediction: cat; Actual: cat
1/1 ━━━━━━━━━━━━━━━

## Try to predict dogs

In [22]:
root_path = 'dataset/test_set/dogs'
files = get_files_by_extension(root_path, file_extension)

try_predict_animals_from_files(files)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
File: dataset\test_set\dogs\dog.4653.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: dataset\test_set\dogs\dog.4313.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
File: dataset\test_set\dogs\dog.4543.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
File: dataset\test_set\dogs\dog.4456.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: dataset\test_set\dogs\dog.4065.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
File: dataset\test_set\dogs\dog.5000.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
File: dataset\test_set\dogs\dog.4896.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
File: dataset\test_set\dogs\dog.4415.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
File: dataset\test_set\dogs\dog.4477.jpg; Prediction: dog; Actual: dog
1/1 ━━━━━━━━━━━━━━━